## 验证集与测试集
利用已标注数据进行深度学习模型的训练，某种程度上类似于“题海战术”——将有标准答案的题目交给计算机，让计算机据此训练深度学习模型，然后将训练好的模型应用于未标注的数据。      
但是一个成熟的题海战术，绝不是简单地把所有题目和答案都告诉计算机，让计算机依据所有的标注数据去训练模型，然后将训练出来的模型直接应用于真实的未标注数据（这会导致我们无法掌握模型的预测正确率）。常用的成熟的做法是将有答案的题目分成两部分：预留一部分有标准答案的题目（验证集），只给计算机另外一部分（训练集）去训练模型，然后将训练好的模型应用在预留的题目上，检验训练得到的模型的准确率，召回率等指标。这种方式下，虽然模型所依据的训练数据少了一些，但我们可以依据模型在验证集上的正确率来预估模型面对真实数据时的预测正确率。     
如何划分题目为训练集和验证集，就是本节学习的第一个内容。
就本次竞赛的题目而言，已经预先划分了训练集和测试集，但这并不表示我们必须把训练集当训练集，然后去验证集上作验证。主要原因是：如果训练集和验证集的数据分布不太一致，则会导致训练出来的模型的预测成绩失真。
为了解决这个问题，一般来说有以下几种划分测试集和验证集的方法：


###  留出法（Hold-Out）
像本次赛题这样直接将训练集划分成训练集和验证集两部分，就是留出法。这种划分方式的优点是最为直接简单；缺点是只得到了一份验证集，有可能导致模型在验证集上过拟合。特别是当测试集和验证集的额分布的不一致性比较大时，将 会产生更加不准确的预测。    
留出法应用场景是数据量比较大的情况。

### 交叉验证（Cross Validation，CV）
#### K折交叉验证
一个比较好的方式是， 我们将赛题给定的训练集和验证集合并，然后再划分成K份，将其中的K-1份作为训练集，剩余的1份作为验证集，循环K训练。这种划分方式是所有的训练集都是验证集，最终模型验证精度是K份平均得到。这种方式的优点是验证集精度比较可靠，训练K次可以得到K个有多样性差异的模型，缺点是需要训练K次，不适合数据量很大的情况。
#### 简单交叉验证
简单的交叉验证的步骤如下：
* 从全部的训练数据中随机选择一部分样例作为训练集，剩余的作为验证集。
* 通过对测试集训练 ，得到模型 。
* 在验证集对每一个样本根据模型进行预测，并于标准答案对比，得到模型再预测集上的正确率等指标。
* 根据评分标准（例如本次赛题 是预测正确率）选择具有最好成绩的模型。
这种方法称为 hold -out cross validation 或者称为简单交叉验证。和K这交叉验证一样，由于测试集和验证集是分开的，就避免了过拟合的现象
#### 自助采样法（BootStrap）

通过有放回的采样方式得到新的训练集和验证集，每次的训练集和验证集都是有区别的。这种划分方式一般适用于数据量较小的情况。在本次赛题中已经划分为验证集，因此选手可以直接使用训练集进行训练，并使用验证集进行验证精度（当然也可以合并训练集和验证集，自行划分验证集）。

当然这些划分方法是从数据划分方式的角度来讲的，在现有的数据比赛中一般采用的划分方法是留出法和交叉验证法。如果数据量比较大，留出法还是比较合适的。当然任何的验证集的划分得到的验证集都是要保证训练集-验证集-测试集的分布是一致的，所以如果不管划分何种的划分方式都是需要注意的。这里的分布一般指的是与标签相关的统计分布，比如在分类任务中“分布”指的是标签的类别分布，训练集-验证集-测试集的类别分布情况应该大体一致；如果标签是带有时序信息，则验证集和测试集的时间间隔应该保持一致。
### 留一法  leave-one-out cross validation
留一法就是每次只留下一个样本做测试集，其它样本做训练集，如果有k个样本，则需要训练k次，测试k次。
留一发计算最繁琐，但样本利用率最高。适合于小样本的情况。就本次赛题而言，由于数据量相对较大，留一法会导致训练周期及其漫长。

In [ ]:
# 合并训练集和验证集
full_path = train_path.extend(val_path) # 虽然文件名有重复，但加上 路径后就具有唯一性，可以直接合并
full_label = train_label.extend(val_label) # 不能直接合并，需将 读取道德JSON文件的key换为包含路径的文件名。
# 加载合并后的集和
full_loader = torch.utils.data.DataLoader(
    SVHNDataset(full_path, full_label,
                transforms.Compose([
                    transforms.Resize((64, 128)),#缩放到固定尺寸
                    transforms.RandomCrop((60, 120)),
                    transforms.ColorJitter(0.3, 0.3, 0.2),#随机颜色变换
                    transforms.RandomRotation(10),#加入随机旋转
                    transforms.ToTensor(),#将图片转换为pytorch 的 tensor
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])#对图像像素进行归一化
    ])), 
    batch_size=40, 
    shuffle=True, 
    num_workers=10, # 设置线程数， win下要注释掉
)

In [ ]:
# 一个10折交叉 验证的pytorch的简单实现
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset   
import torch.nn.functional as F
from torch.autograd import Variable
 
 
 
#####构造的训练集####
x = torch.rand(100,28,28) 
y = torch.randn(100,28,28)
x = torch.cat((x,y),dim=0)
label =[1] *100 + [0]*100  
label = torch.tensor(label,dtype=torch.long)
 
######网络结构##########
class Net(nn.Module):
    #定义Net
    def __init__(self):
        super(Net, self).__init__() 
     
        self.fc1   = nn.Linear(28*28, 120) 
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 2)
  
    def forward(self, x):
       
        x = x.view(-1, self.num_flat_features(x)) 
      
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x)) 
        x = self.fc3(x) 
        return x
    def num_flat_features(self, x):
        size = x.size()[1:] 
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
 
##########定义dataset##########
class TraindataSet(Dataset):
    def __init__(self,train_features,train_labels):
        self.x_data = train_features
        self.y_data = train_labels
        self.len = len(train_labels)
    
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    def __len__(self):
        return self.len
    
    
########k折划分############        
def get_k_fold_data(k, i, X, y):  ###此过程主要是步骤（1）
    # 返回第i折交叉验证时所需要的训练和验证数据，分开放，X_train为训练数据，X_valid为验证数据
    assert k > 1
    fold_size = X.shape[0] // k  # 每份的个数:数据总条数/折数（组数）
    
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)  #slice(start,end,step)切片函数
        ##idx 为每组 valid
        X_part, y_part = X[idx, :], y[idx]
        if j == i: ###第i折作valid
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat((X_train, X_part), dim=0) #dim=0增加行数，竖着连接
            y_train = torch.cat((y_train, y_part), dim=0)
    #print(X_train.size(),X_valid.size())
    return X_train, y_train, X_valid,y_valid
 
 
def k_fold(k, X_train, y_train, num_epochs=3,learning_rate=0.001, weight_decay=0.1, batch_size=5):
    train_loss_sum, valid_loss_sum = 0, 0
    train_acc_sum ,valid_acc_sum = 0,0
    
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train) # 获取k折交叉验证的训练和验证数据
        net =  Net()  ### 实例化模型
        ### 每份数据进行训练,体现步骤三####
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate,\
                                   weight_decay, batch_size) 
       
        print('*'*25,'第',i+1,'折','*'*25)
        print('train_loss:%.6f'%train_ls[-1][0],'train_acc:%.4f\n'%valid_ls[-1][1],\
              'valid loss:%.6f'%valid_ls[-1][0],'valid_acc:%.4f'%valid_ls[-1][1])
        train_loss_sum += train_ls[-1][0]
        valid_loss_sum += valid_ls[-1][0]
        train_acc_sum += train_ls[-1][1]
        valid_acc_sum += valid_ls[-1][1]
    print('#'*10,'最终k折交叉验证结果','#'*10) 
    ####体现步骤四#####
    print('train_loss_sum:%.4f'%(train_loss_sum/k),'train_acc_sum:%.4f\n'%(train_acc_sum/k),\
          'valid_loss_sum:%.4f'%(valid_loss_sum/k),'valid_acc_sum:%.4f'%(valid_acc_sum/k))
 
 
#########训练函数##########
def train(net, train_features, train_labels, test_features, test_labels, num_epochs, learning_rate,weight_decay, batch_size):
    train_ls, test_ls = [], [] ##存储train_loss,test_loss
    dataset = TraindataSet(train_features, train_labels) 
    train_iter = DataLoader(dataset, batch_size, shuffle=True) 
    ### 将数据封装成 Dataloder 对应步骤（2）
    
    #这里使用了Adam优化算法
    optimizer = torch.optim.Adam(params=net.parameters(), lr= learning_rate, weight_decay=weight_decay)
    
    for epoch in range(num_epochs):
        for X, y in train_iter:  ###分批训练 
            output  = net(X)
            loss = loss_func(output,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        ### 得到每个epoch的 loss 和 accuracy 
        train_ls.append(log_rmse(0,net, train_features, train_labels)) 
        if test_labels is not None:
            test_ls.append(log_rmse(1,net, test_features, test_labels))
    #print(train_ls,test_ls)
    return train_ls, test_ls
 
def log_rmse(flag,net,x,y):
    if flag == 1: ### valid 数据集
        net.eval()
    output = net(x)
    result = torch.max(output,1)[1].view(y.size())
    corrects = (result.data == y.data).sum().item()
    accuracy = corrects*100.0/len(y)  #### 5 是 batch_size
    loss = loss_func(output,y)
    net.train()
    
    return (loss.data.item(),accuracy)
 
loss_func = nn.CrossEntropyLoss() ###申明loss函
k_fold(10,x,label) ### k=10,十折交叉验证